# Data Cleaning : _From Bronze Layer to Silver Layer_

**Mount the data csv file from the Azure Storage `StorageAccount` Container `Bronze_container`**
***

In [0]:
dbutils.fs.mount(
    source='wasbs://bronze_container@Storage_account.blob.core.windows.net',
    mount_point='/mnt/bronze_container',
    extra_configs={'fs.azure.account.key.Storage_account.blob.core.windows.net': dbutils.secrets.get('databricksScope', 'MyStorageAccountKey')}
)

dbutils.fs.mount(
    source='wasbs://silver_container@Storage_account.blob.core.windows.net',
    mount_point='/mnt/silver_container',
    extra_configs={'fs.azure.account.key.Storage_account.blob.core.windows.net': dbutils.secrets.get('databricksScope', 'MyStorageAccountKey')}
)

In [0]:
dbutils.fs.ls('/mnt')

[FileInfo(path='dbfs:/mnt/bronze/', name='bronze/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/', name='gold/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/', name='silver/', size=0, modificationTime=0)]

**Initialize Spark**
***

In [0]:
spark

**Basic data exploration**
***
- _Reading the data from a csv File_
- _printing the shape of the dataframe_
- _printing the schema of the dataframe_

In [0]:
from pyspark.sql.functions import col, sum, when

- Reading the store csv file in Azure Data Storage Container `Bronze`

In [0]:
csv_file_path = "dbfs:/mnt/bronze/raw_cyber_attack_data.csv"
df = spark.read.option("delimiter",",").option("wholeFile",True).option("multiline",True).option("header",True).option("inferSchema",True).csv(csv_file_path)

In [0]:
print(f"Shape of DataFrame: {df.count()} rows, {len(df.columns)} columns")

Shape of DataFrame: 40000 rows, 25 columns


In [0]:
df.printSchema()

root
 |-- Timestamp: timestamp (nullable = true)
 |-- Source IP Address: string (nullable = true)
 |-- Destination IP Address: string (nullable = true)
 |-- Source Port: integer (nullable = true)
 |-- Destination Port: integer (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- Packet Length: integer (nullable = true)
 |-- Packet Type: string (nullable = true)
 |-- Traffic Type: string (nullable = true)
 |-- Payload Data: string (nullable = true)
 |-- Malware Indicators: string (nullable = true)
 |-- Anomaly Scores: double (nullable = true)
 |-- Alerts/Warnings: string (nullable = true)
 |-- Attack Type: string (nullable = true)
 |-- Attack Signature: string (nullable = true)
 |-- Action Taken: string (nullable = true)
 |-- Severity Level: string (nullable = true)
 |-- User Information: string (nullable = true)
 |-- Device Information: string (nullable = true)
 |-- Network Segment: string (nullable = true)
 |-- Geo-location Data: string (nullable = true)
 |-- Proxy Informati

In [0]:
df.show(5)

+-------------------+-----------------+----------------------+-----------+----------------+--------+-------------+-----------+------------+--------------------+------------------+--------------+---------------+-----------+----------------+------------+--------------+----------------+--------------------+---------------+------------------+-----------------+-------------+--------------+----------+
|          Timestamp|Source IP Address|Destination IP Address|Source Port|Destination Port|Protocol|Packet Length|Packet Type|Traffic Type|        Payload Data|Malware Indicators|Anomaly Scores|Alerts/Warnings|Attack Type|Attack Signature|Action Taken|Severity Level|User Information|  Device Information|Network Segment| Geo-location Data|Proxy Information|Firewall Logs|IDS/IPS Alerts|Log Source|
+-------------------+-----------------+----------------------+-----------+----------------+--------+-------------+-----------+------------+--------------------+------------------+--------------+--------

In [0]:
for c in df.columns:
    null_count = df.select(sum(when(col(c).isNull(), 1).otherwise(0)).alias("null_count")).collect()[0][0]
    print(f"{c:<30} {null_count}")

Timestamp                      0
Source IP Address              0
Destination IP Address         0
Source Port                    0
Destination Port               0
Protocol                       0
Packet Length                  0
Packet Type                    0
Traffic Type                   0
Payload Data                   0
Malware Indicators             20000
Anomaly Scores                 0
Alerts/Warnings                20067
Attack Type                    0
Attack Signature               0
Action Taken                   0
Severity Level                 0
User Information               0
Device Information             0
Network Segment                0
Geo-location Data              0
Proxy Information              19851
Firewall Logs                  19961
IDS/IPS Alerts                 20050
Log Source                     0


**Silver Layer : handling NaN values**
***

- Replacing any null values in the `Alerts/Warnings` column into **No** and the existing ones into **Yes**

In [0]:
df = df.withColumn('Alerts/Warnings', when(df['Alerts/Warnings'] == 'Alert Triggered', 'yes').otherwise('no'))

- Replacing any null values in the `Malware Indicators` column with **No Detection**

In [0]:
df = df.withColumn('Malware Indicators', when((col('Malware Indicators').isNull()) | (col('Malware Indicators') == ''), 'No Detection').otherwise(col('Malware Indicators'))
)

- Replacing any null values in the `Firewall Logs` column with **No Data**

In [0]:
df = df.withColumn('Firewall Logs', when((col('Firewall Logs').isNull()) | (col('Firewall Logs') == ''), 'No Data').otherwise(col('Firewall Logs'))
)

- Replacing any null values in the `IDS/IPS Alerts` column with **No Data**

In [0]:
df = df.withColumn('IDS/IPS Alerts', when((col('IDS/IPS Alerts').isNull()) | (col('IDS/IPS Alerts') == ''), 'No Data').otherwise(col('IDS/IPS Alerts'))
)

- Replacing any null values in the `Proxy Information` column with **No Data**

In [0]:
df = df.withColumn('Proxy Information', when((col('Proxy Information').isNull()) | (col('Proxy Information') == ''), 'No Proxy').otherwise(col('Proxy Information'))
)

- Checking one more time the existance of the null values

In [0]:
print('Checking the null values ...')
for c in df.columns:
    null_count = df.select(sum(when(col(c).isNull(), 1).otherwise(0)).alias("null_count")).collect()[0][0]
    print(f"{c:<30} {null_count}")

Checking the null values ...
Timestamp                      0
Source IP Address              0
Destination IP Address         0
Source Port                    0
Destination Port               0
Protocol                       0
Packet Length                  0
Packet Type                    0
Traffic Type                   0
Payload Data                   0
Malware Indicators             0
Anomaly Scores                 0
Alerts/Warnings                0
Attack Type                    0
Attack Signature               0
Action Taken                   0
Severity Level                 0
User Information               0
Device Information             0
Network Segment                0
Geo-location Data              0
Proxy Information              0
Firewall Logs                  0
IDS/IPS Alerts                 0
Log Source                     0


- Storing the Processed Data into the `Silver` Container in a csv format

In [0]:
silver_container_path = "/mnt/silver/silver_cyber_attack_data.csv"
df.write.mode("overwrite").csv(silver_container_path, header=True)